In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [8]:
import pandas as pd
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go


In [10]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [36]:
file1 = "/gdrive/My Drive/Final-Project---Netflix-Ratings/data/output/full1.txt"
file2 = "/gdrive/My Drive/Final-Project---Netflix-Ratings/data/output/full2.txt"
file3 = "/gdrive/My Drive/Final-Project---Netflix-Ratings/data/output/full3.txt"
file4 = "/gdrive/My Drive/Final-Project---Netflix-Ratings/data/output/full4.txt"


df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)
df3 = pd.read_csv(file3)
df4 = pd.read_csv(file4)



In [11]:
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1AfIhBgmv-lTvn4Ys00WmO016VK51geV9vAqLrTX3-sY/edit#gid=1955784673')

In [13]:
sheet = wb.worksheet('movie_titles')

In [29]:
data = sheet.get_all_values()
df_Movie = pd.DataFrame(data)
df_Movie.columns = df_Movie.iloc[0]
df_Movie = df_Movie.iloc[1:]

In [30]:
df_Movie.head()

,Id,Year,Name
1,1,2003,Dinosaur Planet
2,2,2004,Isle of Man TT 2004 Review
3,3,1997,Character
4,4,1994,Paula Abdul's Get Up & Dance
5,5,2004,The Rise and Fall of ECW


In [37]:
df = df1
df = df.append(df2)
df = df.append(df3)
df = df.append(df4)
df.head()

,Cust_Id,Rating,Movie_Id
0,1488844,3.0,1
1,822109,5.0,1
2,885013,4.0,1
3,30878,4.0,1
4,823519,3.0,1


In [45]:
df.describe()

,Cust_Id,Rating,Movie_Id
count,1.004805e+08,1.004805e+08,1.004805e+08
mean,1.322489e+06,3.604290e+00,2.219795e+03
std,7.645368e+05,1.085219e+00,1.271638e+03
min,6.000000e+00,1.000000e+00,1.000000e+00
25%,6.611980e+05,3.000000e+00,1.146000e+03
50%,1.319012e+06,4.000000e+00,2.173000e+03
75%,1.984455e+06,4.000000e+00,3.290000e+03
max,2.649429e+06,5.000000e+00,4.711000e+03


In [47]:
df_Movie.describe()

,Id,Year,Name
count,17770,17770,17770
unique,17770,95,17360
top,6537,2004,The Hunchback of Notre Dame
freq,1,1436,6


Remove Movies That weren't popular and remove customers that aren't very active

In [38]:
data = df['Rating'].value_counts().sort_index(ascending=False)

trace = go.Bar(x = data.index,
               text = ['{:.1f} %'.format(val) for val in (data.values / df.shape[0] * 100)],
               textposition = 'auto',
               textfont = dict(color = '#000000'),
               y = data.values,
               marker = dict(color = '#33cc99'))
# Create layout
layout = dict(title = 'Distribution Of {} Netflix-Ratings'.format(df.shape[0]),
              xaxis = dict(title = 'Rating'),
              yaxis = dict(title = 'Count'))
# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [34]:
data = df_Movie['Year'].value_counts().sort_index()

# Create trace
trace = go.Scatter(x = data.index,
                   y = data.values,
                   marker = dict(color = '#33cc99'))
# Create layout
layout = dict(title = '{} Movies Grouped By Year Of Release'.format(df_Movie.shape[0]),
              xaxis = dict(title = 'Release Year'),
              yaxis = dict(title = 'Movies'))

# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [49]:
# Get data
data = df.groupby('Movie_Id')['Rating'].count().clip(upper=9999)

# Create trace
trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0,
                                  end = 10000,
                                  size = 100),
                     marker = dict(color = '#db0000'))
# Create layout
layout = go.Layout(title = 'Distribution Of Ratings Per Movie (Clipped at 9999)',
                   xaxis = dict(title = 'Ratings Per Movie'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)



##### Ratings Per User #####
# Get data
data = df.groupby('Cust_Id')['Rating'].count().clip(upper=199)

# Create trace
trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0,
                                  end = 200,
                                  size = 2),
                     marker = dict(color = '#db0000'))
# Create layout
layout = go.Layout(title = 'Distribution Of Ratings Per User (Clipped at 199)',
                   xaxis = dict(title = 'Ratings Per User'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
df.to_csv("/gdrive/My Drive/Final-Project---Netflix-Ratings/data/output/combined.txt",index = False)